# Self-Driving Car Engineer Nanodegree


## Project 1: **Finding Lane Lines on the Road** 
---

## 1. Purpose
** The purpose of the project is to demonstrate my ability to apply the knowledge and technique learned during this course **



## 2. Approach
** Lane detection is a complicated problem under different light/weather conditions. In this project, I am going to apply the Gaussian Blur, Canny edge detection and Hough Transform to detect lane lines on the road.**

### (A.) 1st Pipeline
** The 1st step for this lane detection is to convert images to grayscale, then apply Gaussian Blur to smooth the images. I used the kernel size of 7 to reduce more noises. I then applied the Canny edge detection to detect lane lines on the road.  Typically, the ratio between low and high threshold is either 1:2 or 1:3.  I chose the low threshold = 30 and high threshold = 100 because this gave me the expected results.  
I then chose the trapezoid shape as the region of interest and applied the mask to filter out the region which I want to apply Hough transform algorithm to detect lines. There are more than one function in OpenCV to use for Hough transform, but I chose HoughLinesP probabilistic function because this will be much faster and efficient.  Below ae the parameters which I chose to get the correct lane detection for the tested images:**

`rho = 0.3, `

`theta = np.pi/180, `

`threshold = 12, `

`min_line_len = 5, `

`max_line_gap = 10. `

**After applying Hough Transform, this gave me a set of lines and I can use the typical draw line function in OpenCV to draw lines to detect lanes on road.**

### Results:
** Here are the results to detect lane lines of the two of the six images in the provided `test_image` folder in the 1st pipeline.  As you can see the continuous solid lane lines are detected with the solid red lines whereas the spaces lane lines on road are detected with the dotted red line: **

<figure> 
 <img src="1.png" width="380" alt="Combined Image" />
 <img src="2.png" width="380" alt="Combined Image" />
</figure>

** Now apply this technique to detect lane lines in the two provided videos: "solidWhiteRight" and "solidYellowLeft" **


<figure> 
 <img src="myWhite.png" width="380" alt="Combined Image" />
 <img src="myYellow.png" width="380" alt="Combined Image" />
</figure>

---

### (B.) 2nd Pipeline - The Improved Draw Line Function

** As you can see even though the 1st pipeline gave correct lane detection on the road, it didn't give a smooth solid lane detection.  To improve this lane line detection technique, I need to rewrite the draw line function in the following steps: **

** 1. For each line from the Hough Transform, I need to calculate its slope and based on whether it is negative or positive value I separate it in either two different list. **

```
for x1, y1, x2, y2 in line:
   slope = float((y2-y1)/(x2-x1))
   if (slope > 0):
      positive_slope_lines.append([x1,y1,x2,y2])
   elif (slope < 0):
      negative_slope_lines.append([x1,y1,x2,y2])
```


** 2. I then average the position of each of the lines and extrapolate to the top and bottom of the lane for each of the positive and negative slope lines.  By observation and calculation, the positive slope lines are correspondent to the right lane and the negative slope lines are correspondent to the left lane.**

```
# Average the position of the positive and negative slope lines.
    positive_slope_points = np.average(positive_slope_lines, axis=0)
    negative_slope_points = np.average(negative_slope_lines, axis=0)
    
# extract coordinate points for each positive and negative slope lines
    upper_rightX, upper_rightY, lower_rightX, lower_rightY = positive_slope_points
    lower_leftX, lower_leftY, upper_leftX, upper_leftY = negative_slope_points
```

** 3. I calculate the positive and negative slope of the extrapolated lines.  Given the region of interest and the above slope of the lines, I then determine the coordinate/location of the remain points of interest, and draw the lines.  These lines are the lane line detection which gave a very solid line detect for both left and right lanes.**

```
# Calculate the positive and negative slope:
pos_slope = ((lower_rightY - upper_rightY)/(lower_rightX - upper_rightX))
neg_slope = ((upper_leftY - lower_leftY)/(upper_leftX - lower_leftX))

#find the coordinate of the remain point of interest and draw a line:
#for right lane:
pos_y = int(pos_slope * (img.shape[1] - upper_rightX) + upper_rightY)
cv2.line(img,tuple(vertices[0][2]), (img.shape[1],pos_y), color, thickness)

#for left lane:
neg_y = int(neg_slope * (0 - lower_leftX) + lower_leftY)
cv2.line(img,tuple(vertices[0][1]),(0,neg_y), color, thickness)    

```

### Results:
** Here are the results to detect lane lines of the three of the six images in the provided `test_image` folder using the improved draw line technique which I discussed above.  As you can see this technique gave a single and very solid for both left and right lane.**



<figure> 
 <img src="1a.png" width="380" alt="Combined Image" />
 <img src="2a.png" width="380" alt="Combined Image" />
 <img src="3a.png" width="380" alt="Combined Image" />
</figure>

** This technique also improve the lane detection in videos as well as you can you below:**

<figure> 
 <img src="myWhite_enhanced.png" width="380" alt="Combined Image" />
 <img src="myYellow_enhanced.png" width="380" alt="Combined Image" />
</figure>

## 3. Reflection
** Details of my code pipeline have been discussed in the sections Approach above. **

** This Hough transform with the enhanced draw line technique based on the slope indeed gave me a very solid lane detection on the road under some certain conditions: empty or very light traffic road, straight road, no road structure changes, and no shadow of the tree around.  However, under some other conditions like roads with lots of trees or shadow of the trees on the roads, this technique start to fail.**

** If such conditions occur, take these steps to tune to improve the lane detection: **

** 1. Apply Gaussian blur with much bigger kernel to further smooth the images. This could help under this "shortcoming" scenarios which I mentioned above**

**2. Tune the low and high threshold of the Canny edge detection could also play a major role in improving the lane detect condition.**

**3. Color normalization, Erode and Dilate technique can also be used to tune and prevent the "shortcoming" scenarios. **

## 4. Optional Challenge
** In this section, the road structure changes and adds more unwanted features on it such as the tree shadows, freeway divider, and the surface of the freeway road shows unwanted lines and patterns.  This will cause my current tuning parameters detect unwanted features as well, as shown below. **

<figure> 
 <img src="challenge.png" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> The algorithm failed to pick up unwanted freeway divider, the car hood, and trees </p> 
 </figcaption>
 
 <img src="challengetree.png" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> The algorithm failed to pick unwanted tree shadow on freeway road </p> 
 </figcaption>

</figure>

** In this scenario, by applying bigger filter for Gaussian Blur to smooth the images, this actually helps to reduce the unwanted features as shown below**

``` kernel_size = 23
low_threshold = 60
high_threshold = 100    
rho = 0.3
theta = np.pi/180
threshold = 10
min_line_len = 20
max_line_gap = 5
vertices = np.array([[(169,653), (497,423), (718,423), (1049,653)]],dtype=np.int32)```


<figure> 
 <img src="challenge_filter.png" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Apply bigger kernel to filter out unwanted features </p> 
 </figcaption>
 </figure>
 